In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture

# Joblib
!pip install joblib

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import joblib
import pickle

# Load data

In [4]:
TRAIN_PATH = '/content/drive/MyDrive/Public/CS221 - Natural Language Processing/uit_vsmec_processed/train_processed.csv'
VAL_PATH = '/content/drive/MyDrive/Public/CS221 - Natural Language Processing/uit_vsmec_processed/val_processed.csv'
TEST_PATH = '/content/drive/MyDrive/Public/CS221 - Natural Language Processing/uit_vsmec_processed/test_processed.csv'

MODELS_PATH = '/content/drive/MyDrive/Public/CS221 - Natural Language Processing/models/'
RESULTS_PATH = '/content/drive/MyDrive/Public/CS221 - Natural Language Processing/results/'

In [5]:
GROUP_1 = ['Anger', 'Disgust', 'Sadness', 'Fear'] # 0
GROUP_2 = ['Enjoyment', 'Surprise', 'Other'] # 1

In [6]:
def transform_y(y):
    return y.map(lambda i: 0 if i in GROUP_1 else 1)

def load_data(path):
    data = pd.read_csv(path)
    data.cleaned_sentence.fillna('', inplace=True)
    X = data.cleaned_sentence
    y = data.emotion
    y = transform_y(y)

    return X, y

In [7]:
X_train, y_train = load_data(TRAIN_PATH)
X_val, y_val = load_data(VAL_PATH)
X_test, y_test = load_data(TEST_PATH)

# Machine Learning

* Feature Extraction: TF-IDF
* Models:
    - Logistic Regression
    - SVM
    - Decision Tree
    - Random Forest
    - kNN
    - Naive Bayes

In [8]:
# Machine Learning
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

## TF-IDF

In [9]:
# load from baseline
with open(MODELS_PATH + 'baseline/tfidf.pkl', 'rb') as f:
    tfidf = pickle.load(f)

In [10]:
X_train_tfidf = tfidf.transform(X_train)
X_val_tfidf = tfidf.transform(X_val)
X_test_tfidf = tfidf.transform(X_test)

## Training ML Models


In [11]:
def build_model_ML(model_name):
    if model_name == 'lr':
        clf = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000, verbose=True)
    elif model_name == 'svc':
        clf = SVC(verbose=True,)
    elif model_name == 'dt':
        clf = DecisionTreeClassifier()
    elif model_name == 'rf':
        clf = RandomForestClassifier(n_estimators=150, random_state=0)
    elif model_name == 'knn':
        clf = KNeighborsClassifier(n_neighbors=15)
    elif model_name == 'nb':
        clf = MultinomialNB()
    else:
        print('model_name error~!')
        return None

    return clf

In [12]:
# ML
lr = build_model_ML('lr')
svc = build_model_ML('svc')
dt = build_model_ML('dt')
rf = build_model_ML('rf')
knn = build_model_ML('knn')
nb = build_model_ML('nb')

In [13]:
# LR
lr.fit(X_train_tfidf, y_train)
joblib.dump(lr, MODELS_PATH + 'model_1/lr_model1.h5')

# SVC
svc.fit(X_train_tfidf, y_train)
joblib.dump(svc, MODELS_PATH + 'model_1/svc_model1.h5')

# DT
dt.fit(X_train_tfidf, y_train)
joblib.dump(dt, MODELS_PATH + 'model_1/dt_model1.h5')

# RF
rf.fit(X_train_tfidf, y_train)
joblib.dump(rf, MODELS_PATH + 'model_1/rf_model1.h5')

# kNN
knn.fit(X_train_tfidf, y_train)
joblib.dump(knn, MODELS_PATH + 'model_1/knn_model1.h5')

# NB
nb.fit(X_train_tfidf, y_train)
joblib.dump(nb, MODELS_PATH + 'model_1/nb_model1.h5')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


[LibSVM]

['/content/drive/MyDrive/Public/CS221 - Natural Language Processing/models/model_1/nb_model1.h5']

# Evaluation

In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [15]:
lr = joblib.load(MODELS_PATH + 'model_1/lr_model1.h5')
svc = joblib.load(MODELS_PATH + 'model_1/svc_model1.h5')
dt = joblib.load(MODELS_PATH + 'model_1/dt_model1.h5')
rf = joblib.load(MODELS_PATH + 'model_1/rf_model1.h5')
knn = joblib.load(MODELS_PATH + 'model_1/knn_model1.h5')
nb = joblib.load(MODELS_PATH + 'model_1/nb_model1.h5')

In [16]:
def evaluate(model, X, y):
    preds = model.predict(X)

    acc = round(accuracy_score(y, preds), 2)
    pre = round(precision_score(y, preds), 2)
    recall = round(recall_score(y, preds), 2)
    f1 = round(f1_score(y, preds), 2)

    return [acc, pre, recall, f1]

In [17]:
def get_result(X, y):
    lr_re = evaluate(lr, X, y)
    svc_re = evaluate(svc, X, y)
    dt_re = evaluate(dt, X, y)
    rf_re = evaluate(rf, X, y)
    knn_re = evaluate(knn, X, y)
    nb_re = evaluate(nb, X, y)

    re = pd.DataFrame(
        [lr_re] + [svc_re] + [dt_re] + [rf_re] + [knn_re] + [nb_re],
        columns=['accuracy', 'precision', 'recall', 'f1'],
        index=['lr', 'svc', 'dt', 'rf', 'knn', 'nb']
    )

    return re

In [18]:
re_train = get_result(X_train_tfidf, y_train)
re_train

,accuracy,precision,recall,f1
lr,0.85,0.84,0.86,0.85
svc,0.98,0.98,0.97,0.98
dt,1.00,1.00,1.00,1.00
rf,1.00,1.00,1.00,1.00
knn,0.77,0.78,0.77,0.78
nb,0.86,0.87,0.85,0.86


In [19]:
re_val = get_result(X_val_tfidf, y_val)
re_val.to_csv(RESULTS_PATH + 'model1_val_ml.csv')
re_val

,accuracy,precision,recall,f1
lr,0.75,0.81,0.73,0.77
svc,0.77,0.83,0.75,0.79
dt,0.65,0.71,0.63,0.67
rf,0.71,0.79,0.66,0.72
knn,0.72,0.80,0.68,0.73
nb,0.74,0.82,0.69,0.75


In [20]:
re_test = get_result(X_test_tfidf, y_test)
re_test.to_csv(RESULTS_PATH + 'model1_test_ml.csv')
re_test

,accuracy,precision,recall,f1
lr,0.76,0.76,0.78,0.77
svc,0.75,0.75,0.77,0.76
dt,0.61,0.64,0.57,0.61
rf,0.73,0.76,0.71,0.73
knn,0.69,0.72,0.67,0.69
nb,0.73,0.75,0.72,0.73
